# BUILD THE NEURAL NETWORK

神经网络由对数据执行操作的层/模块组成。其中[torch.nn](https://pytorch.org/docs/stable/nn.html)命名空间提供了你搭建自己的神经网络所需要的所有构建模块。所有PyTorch中的模块都是[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)的子类。神经网络是一个由其他模块（层）组成的模块本身。这种网络结构使得我们可以简单的搭建和管理复杂的结构。

在这一节中，我们将搭建一个神经网络来分类FashionMNIST数据集中的图片。

In [40]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

如果可以的话，我们想要通过像GPU或MPS等硬件来加速我们的训练。让我们来检查一下[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)或[torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html)是否可用，否则我们就使用CPU。

In [2]:
device = (
    "cuda" 
    if torch.cuda.is_available() 
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## Define the Class

我们通过继承`nn.Module`来定义我们的神经网络，并且在`__init__`中初始化神经网络层。每个`nn.Module`子类在`forward`函数中实现对输入数据的操作。

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建了一个`NeuralNetwork`的实例，并且将它放到`device`中，并打印它的结构。

In [15]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


为了使用这个模型，我们把输入数据传给它。这就执行了模型的`forward`，以及一些后台操作。不要直接调用`model.forward()`!

在输入上调用模型会返回一个二维张量，dim=0对应于每个类的10个原始预测值的每个输出，dim=1对应于每个输出的单个值。我们通过`nn.Softmax`模块的一个实例来获得预测概率。

In [24]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


## Model Layers

让我们把FashionMNIST模型逐层分解。为了说明，我们拿一个包含3张28×28图片的minibatch,并将它传给网络看看发生了什么。

In [32]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

我们初始化[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)层把每个2D 28×28的图像转换为一个有784个像素值的连续的数组。

In [31]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
[线性层](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)是对输入应用线性变换的一个模块，并存储权重和偏置值。

In [33]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

非线性激活用于在模型输入和输出之间创建一个复杂的映射。它们被应用在线性变换之后，以引入非线性，帮助神经网络学习各种各样的现象。

在这个模型中，我们在线性层之间使用了[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html),但还有其他激活方式可以在你的模型中引入非线性。

In [35]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.2936, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1140,
         0.0000, 0.1780, 0.2936, 0.0000, 0.0000, 0.3558, 0.0000, 0.0000, 0.0000,
         0.3938, 0.2266],
        [0.3446, 0.0000, 0.0000, 0.0175, 0.0000, 0.0000, 0.0292, 0.0000, 0.3856,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2926, 0.1275, 0.0000, 0.0000,
         0.1042, 0.0875],
        [0.0102, 0.1174, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2630, 0.1790,
         0.0000, 0.2135, 0.0000, 0.0000, 0.0000, 0.4785, 0.0000, 0.0000, 0.0000,
         0.2369, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.2936, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1140,
         0.0000, 0.1780, 0.2936, 0.0000, 0.0000, 0.3558, 0.0000, 0.0000, 0.0000,
         0.3938, 0.2266],
        [0.3446, 0.0000, 0.0000, 0.0175, 0.0000, 0.0000, 0.0292, 0.0000, 0.3856,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2926, 0.1275, 0.0000, 0.0000,
         0.1042, 0.0875],
       

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)是一个模块的顺序容器。数据会以定义的顺序通过所有模块。你可以使用顺序容器来拼凑一个快速网络，如`seq_modules`。

In [36]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax

神经网络的最后一个线性层返回对数--\[-infty, infty\]中的原始值--并传递给[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)模块。对数被缩放为数值\[0, 1\]，代表模型对每个类别的预测概率。 `dim`参数表示数值必须和为1的维度。

In [37]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model Parameters

神经网络中的许多层都是参数化的，即有相关的权重和偏置，在训练中得到优化。子类化  `nn.Module` 会自动跟踪所有在你的模型对象中定义的字段，并使所有的参数可以使用你的模型的 `parameters()` 或 `named_parameters()` 方法来访问。

In [39]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0041, -0.0084,  0.0075,  ..., -0.0067,  0.0141,  0.0164],
        [ 0.0153, -0.0149, -0.0179,  ..., -0.0019, -0.0218, -0.0242]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0037, -0.0047], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0362,  0.0146, -0.0260,  ...,  0.0418, -0.0293,  0.0127],
        [-0.0407,  0.0365,  0.0286,  ..., -0.0346,  0.0262, -0.0191]],
       device='cuda:0', grad_fn=<SliceBa

## Further Reading
- [torch.nn API](https://pytorch.org/docs/stable/nn.html)